## Testing model

In [ ]:
import os
from os import path
from glob import iglob
from PIL import Image, ImageOps
import numpy as np
import base64
import json
from io import BytesIO
import sys
import lightgbm as lgb
import shutil
from datetime import datetime
import boto3
import io

import keras
from tensorflow.keras.applications.resnet50 import ResNet50
from keras.preprocessing import image
from keras.applications.imagenet_utils import preprocess_input

os.environ["KERAS_BACKEND"] = "cntk"

In [ ]:
# Name of your bucket with images
bucket_name = 'bucketwithvideo'

In [ ]:
session = boto3.session.Session()

ENDPOINT = "https://storage.yandexcloud.net"

session = boto3.Session(
    aws_access_key_id=(os.environ['token']),
    aws_secret_access_key=(os.environ['key_value']),
    region_name="ru-central1",
)


s3 = session.client(
    "s3", endpoint_url=ENDPOINT)

def from_s3(bucket, key):
    file_byte_string = s3.get_object(Bucket=bucket, Key=key)['Body'].read()
    img = Image.open(io.BytesIO(file_byte_string))
    return img

In [ ]:
test_image = 'car/car-17.jpeg'
#test_image = 'bus/bus-183.jpeg'

In [ ]:
fail_pil_image = from_s3('smartsmart', test_image)

print(fail_pil_image.size)
fail_pil_image

## Prediction

In [ ]:
%%time
clf = lgb.Booster(model_file='lightgbm_classifier.model')
model = ResNet50(weights='imagenet',  input_shape=(224, 224, 3))
img = from_s3('smartsmart', test_image).resize((224, 224))
x = image.img_to_array(img)
x = np.expand_dims(x, axis=0)
x = preprocess_input(x)
pred = model.predict(x).squeeze()  
feat = pred.reshape(1, pred.shape[0])
resp = clf.predict(feat)
resp